<a href="https://colab.research.google.com/github/Learnheart/URL-Classification/blob/main/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 54.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


1.2.2


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from keras.utils import to_categorical
from keras.models import Sequential
# from keras.layers.core import Dense, Activation
from keras.layers import Dense, Activation
# from keras.utils import np_utils
import re
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier


!pip3 install pyvi
from pyvi import ViUtils

In [ ]:
import sklearn
print(sklearn.__version__)

1.4.1.post1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
names=['URL','Category']
df=pd.read_csv('/content/drive/MyDrive/NCKH/Data/full training data.csv',names=names, usecols=[0, 2], na_filter=False,  encoding='latin-1')
print(df.shape)
df[10632:10640]

(379119, 2)


,URL,Category
10632,bonbanh.com/ha-noi/oto/suzuki-grand_vitara-nam...,Automotive
10633,bonbanh.com/oto/mitsubishi-mirage-so-san-cu-da...,Automotive
10634,"mitsubishibacquang.bonbanh.com/chi-tiet-xe/id,...",Automotive
10635,kienthuc.net.vn/o-to-xe-may/nhung-quy-dinh-moi...,Automotive
10636,bonbanh.com/ha-noi/oto/mitsubishi-sf000000020/...,Automotive
10637,bonbanh.com/xe-porsche-cayenne--2008-3751103,Automotive
10638,otofun.net/threads/hoi-kinh-nghiem-dung-bong-d...,Automotive
10639,bonbanh.com/oto/toyota-corolla_altis-sedan-4-c...,Automotive


#Pre-processing

In [ ]:
def url_to_text(url):
  # remove stopwords
  url = url.replace('.html','').replace('.htm','').replace('http://','').replace('https://','')
  url = re.sub('^(.*?/)','/', url) # remove domains
  url = re.sub('[0-9]+', '', url)
  url = re.sub('[_\-/]+', ' ', url)

  return ViUtils.remove_accents(url.lower()).decode()

# df['URL'] = df['URL'].apply(url_to_text)
# df
# dataset = df[:]
# train, test = train_test_split(dataset, random_state=33, test_size=0.1)
# train, val = train_test_split(train, random_state=44, test_size=0.2)
# train.tail()
# val.tail()
# test.tail()

#Building model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import pickle
import time

# Your existing code
X = df['URL'].apply(url_to_text)
y = df['Category']

# Split the data into train, validation, and test sets
train, test = train_test_split(df, random_state=33, test_size=0.1)
train, val = train_test_split(train, random_state=44, test_size=0.2)

# Assuming 'target_column' is the target column in your DataFrame
X_train, y_train = train['URL'].apply(url_to_text), train['Category']
X_val, y_val = val['URL'].apply(url_to_text), val['Category']
X_test, y_test = test['URL'].apply(url_to_text), test['Category']

# Create a pipeline
vectorizer = TfidfVectorizer() #transforms text data into numerical feature vectors
pipeline = Pipeline([ #pipeline contain all compponents as tuple
    ('vect', vectorizer),
    ('chi',  SelectKBest(chi2, k=1200)), #uses the chi-squared (chi2) statistical test to select the k=1200 best features.
    ('clf', RandomForestClassifier())
])

In [ ]:

# Record the start time
start_time = time.time()

# Fit the model
model = pipeline.fit(X_train, y_train)

# Save the model using pickle
# with open('RandomForest.pickle', 'wb') as f:
#     pickle.dump(model, f)

# Record the end time
end_time = time.time()

# Calculate and print the training time
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

# Evaluate the model on the test set
y_test_pred = model.predict(X_test)

# Print the classification report and confusion matrix
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))


Training time: 539.4063894748688 seconds
                        precision    recall  f1-score   support

            Automotive       1.00      0.97      0.99      3108
    Books & Literature       0.88      0.97      0.93      1695
    Business & Finance       0.99      0.99      0.99      2887
               Careers       0.99      0.96      0.97      1415
             Education       0.99      1.00      0.99      1772
   Entertainment & Art       0.98      0.98      0.98      4593
           Environment       0.96      0.83      0.89      1013
Family & Relationships       0.96      0.94      0.95      1530
          Food & Drink       0.92      0.97      0.95      2943
                 Games       0.97      0.94      0.96       984
        Healthy Living       0.96      0.95      0.96      1407
         Home & Garden       0.98      0.99      0.98      1702
       Laws & Policies       0.99      0.90      0.94       995
       News & Politics       0.95      0.96      0.95      220

In [ ]:
new_input_url = 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/'
new_input_processed = url_to_text(new_input_url)

# Make prediction for the new input
model_prediction = model.predict([new_input_processed])

# Print the prediction
print(f"Prediction for '{new_input_url}': {model_prediction}")


Prediction for 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/': ['Healthy Living']


In [ ]:
import joblib
joblib.dump(model, '/content/drive/MyDrive/NCKH/HyperModel/random_forest_1.4.1.pkl')

['/content/drive/MyDrive/NCKH/HyperModel/random_forest_1.4.1.pkl']

In [ ]:

# random fores
rf_prediction = rf_model.predict([new_input_processed])
print(f"Prediction for '{new_input_url}': {rf_prediction}")

NameError: name 'rf_model' is not defined

In [ ]:
test_model = pickle.load(open('/content/drive/MyDrive/NCKH/HyperModel/random_forest_origin_pickle.pkl', 'rb'))
test_model = test_model.predict([new_input_processed])
print(test_model)

['Healthy Living']


#hyper-parameter tuning

In [ ]:
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import logging

param_grid = {
    'clf__n_estimators': [100, 200, 300],  # default 100
    'clf__max_depth': [None, 10, 20],       # default none
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', verbose=2)

# Configure logging
logging.basicConfig(level=logging.INFO)  # Set logging level to INFO

# Fit the grid search to the training data
logging.info("Grid search started...")
grid_search.fit(X_train, y_train)
logging.info("Grid search completed.")

# Accessing cross-validated scores
cv_results = grid_search.cv_results_
mean_test_scores = cv_results['mean_test_score']
params = cv_results['params']

# Logging progress
for mean_score, params in zip(mean_test_scores, params):
    logging.info("Mean Test Score: %f", mean_score)
    logging.info("Hyperparameters: %s", params)


for mean_score, params in zip(mean_test_scores, params):
    print("Mean test score: ", mean_score)
    print("Hyperparameter", params)
    print("-------------------")

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .........clf__max_depth=None, clf__n_estimators=100; total time= 8.4min
[CV] END .........clf__max_depth=None, clf__n_estimators=100; total time= 8.1min
[CV] END .........clf__max_depth=None, clf__n_estimators=100; total time= 8.0min
[CV] END .........clf__max_depth=None, clf__n_estimators=200; total time=16.1min
[CV] END .........clf__max_depth=None, clf__n_estimators=200; total time=15.6min
[CV] END .........clf__max_depth=None, clf__n_estimators=200; total time=15.6min
[CV] END .........clf__max_depth=None, clf__n_estimators=300; total time=23.5min
[CV] END .........clf__max_depth=None, clf__n_estimators=300; total time=22.9min
[CV] END .........clf__max_depth=None, clf__n_estimators=300; total time=23.5min
[CV] END ...........clf__max_depth=10, clf__n_estimators=100; total time=  25.5s
[CV] END ...........clf__max_depth=10, clf__n_estimators=100; total time=  25.9s
[CV] END ...........clf__max_depth=10, clf__n_est

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [ ]:
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=100, cv=3, verbose=2, n_jobs=-1, random_state=42)

# start_time = time.time()
# random_search.fit(X_train, y_train)
# end_time = time.time()

# # Calculate and print the training time
# training_time = end_time - start_time
# print(f"Training time: {training_time} seconds")

# # Print the best parameters and scores
# print(f"Best parameters: {random_search.best_params_}")
# print(f"Best cross-validation score: {random_search.best_score_}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=100. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 9 candidates, totalling 27 fits


KeyboardInterrupt: 

#Testing

In [ ]:

# Load the trained model
# with open('RandomForest.pickle', 'rb') as f:
#     model = pickle.load(f)

# new_input_url = 'https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/song-khoe/cuoc-song-ben-bo-bien-giup-ban-xoa-diu-tam-tri/'
# new_input_processed = url_to_text(new_input_url)

# Make prediction for the new input
prediction = grid_search.predict([new_input_processed])

# Print the prediction
print(f"Prediction for '{new_input_url}': {prediction}")


NameError: name 'grid_search' is not defined

In [ ]:

import joblib
joblib.dump(grid_search, '/content/drive/MyDrive/NCKH/HyperModel/hyper_random_forest.pkl')


In [ ]:

import joblib
# Later, you can load the model from the file
loaded_model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/hyper_random_forest.pkl')
prediction = loaded_model.predict([new_input_processed])

# Print the prediction
print(f"Prediction for '{new_input_url}': {prediction}")

In [ ]:
import seaborn as sns

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=model.classes_, yticklabels=model.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Feature Importance for RandomForestClassifier
if hasattr(model.named_steps['clf'], 'feature_importances_'):
    feature_importances = model.named_steps['clf'].feature_importances_
    feature_names = model.named_steps['vect'].get_feature_names_out()

    # Get top N important features
    top_n = 20
    indices = feature_importances.argsort()[-top_n:][::-1]

    # Plot top N features
    plt.figure(figsize=(10, 6))
    plt.bar(range(top_n), feature_importances[indices], align="center")
    plt.xticks(range(top_n), [feature_names[i] for i in indices], rotation=45)
    plt.title("Top {} Important Features - RandomForestClassifier".format(top_n))
    plt.xlabel("Feature")
    plt.ylabel("Importance")
    plt.show()

# Plot the ROC curve (if applicable, for binary classification)
if len(model.classes_) == 2:
    from sklearn.metrics import plot_roc_curve

    plt.figure(figsize=(8, 6))
    plot_roc_curve(model, X_test, y_test)
    plt.title("ROC Curve")
    plt.show()


In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.
